In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import shutil
import opendatasets as od
from sklearn.preprocessing import LabelEncoder
pd.set_option('mode.chained_assignment', None)
from tqdm import tqdm

In [3]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
subdir = 'utils'
path = os.path.join(parent_dir, subdir)
sys.path.append(path)
from tqdm import tqdm

In [4]:
# Get the data from here https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data
test_df = pd.read_csv('https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv')

train_df = pd.read_csv('https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv')

In [5]:
train_df = train_df.fillna(0)

In [6]:
train_df['Age'].describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [7]:
def is_numeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

# Decision Treest


In [8]:
class Decision_tree_new():
    def __init__(self, data, target, depth) -> None:
        self.data = data
        self.target = target
        self.splits = [('Sex', 'male')]
        self.columns = ['Name', 'Age', 'SibSp']
        self.depth = depth
        self.run_splits()

    def run_splits(self):
        while self.depth >= 1:
            self.new_split()
            self.depth -= 1
        

    def split_to_node(self, condition):
        node_left = self.data[condition].copy(deep=True)
        node_right = self.data[~condition].copy(deep=True)
        return node_left, node_right


    def new_split(self):
        best_score= 0
        best_new_split = None
        for split in self.splits:
            # Go to the branch and look at left and right child
            if is_numeric( split[1]):
                node_left, node_right = self.split_to_node(self.data[split[0]] >= float(split[1]))
            else:
                node_left, node_right = self.split_to_node(self.data[split[0]] == split[1])
            # Calculate best split for left and right child
            left_split, left_score = self.best_slit_for_node(node_left, self.columns)
            right_split, right_score = self.best_slit_for_node(node_right, self.columns)
            # Update best split
            temp_dict = {left_score : left_split, right_score : right_split,  best_score: best_new_split}
            max_key = max(temp_dict.keys())
            # Get the value associated with the maximum key
            best_score = max_key
        new_split = temp_dict[best_score]
        self.splits.append(new_split)
        print(self.splits)


    def best_slit_for_node(self,data, columns):
        self.gini_values = {}
        for col  in columns:
            for val in self.data[col].unique():
                gini_weighted = self.split(data, col, val)
                self.gini_values[col + '_split_' + str(val)] = gini_weighted
        # Find the minimum value of the dictionary and update the self.data
        min_key = min(self.gini_values, key=self.gini_values.get)
        col, val = min_key.split('_split_')
        return (col, val), self.gini_values[min_key]


    def split(self, data, col, val):
        # Calculate probability for each node
        cls1 = data[data[col] == val]
        cls2 = data[data[col] != val]
        prob_list, n1, n2 = self.calculate_probability(cls1, cls2)
        
        # Calculate gini_index for each node
        g1, g2 = self.gini_index(prob_list)
        
        # Return weighted gini-impuritiny index
        return self.gini_impurity_for_split(g1, g2, n1, n2)


    def calculate_probability(self, cls1, cls2):
        ''''''
        prob_cls1 = []
        prob_cls2 = []
        n1 = len(cls1)
        n2 = len(cls2)
        for val in cls1[self.target].unique():
            temp_data = len(cls1[cls1[self.target] == val].copy(deep=True))
            prob_cls1.append(temp_data / n1)

        for val in cls2[self.target].unique():
            value2 = len(cls2[cls2[self.target] == val].copy(deep=True))
            prob_cls2.append(value2 / n2)
        return [prob_cls1, prob_cls2], n1, n2
        

    def gini_index(self, prob_list):
        g1 =  1 - np.sum([element **2 for element in prob_list[0]])
        g2 =  1 - np.sum([element **2 for element in prob_list[1]])
        return g1, g2
        

    def gini_impurity_for_split(self, g1, g2, n1, n2):
        return n1/(n1+n2) * g1 + n2/(n1+n2) * g2
        

In [9]:
target = 'Survived'
n_depth = 5
decision_tree = Decision_tree_new(train_df, target, n_depth)

[('Sex', 'male'), ('Age', '9.0')]
[('Sex', 'male'), ('Age', '9.0'), ('SibSp', '1')]
[('Sex', 'male'), ('Age', '9.0'), ('SibSp', '1'), ('SibSp', '1')]
[('Sex', 'male'), ('Age', '9.0'), ('SibSp', '1'), ('SibSp', '1'), None]


TypeError: 'NoneType' object is not subscriptable